In [2]:
import pandas as pd
import numpy as np
import requests
from scipy.stats import t
import plotly.express as px

In [3]:
odds = pd.read_csv("/media/hdd/tmp/thoroughbred-parsed/thoroughbred-odds-2021.csv", dtype={'market_id': object, 'selection_id': object})

race = pd.read_csv("/media/hdd/tmp/thoroughbred-parsed/thoroughbred-race-data.csv", dtype={'market_id': object, 'selection_id': object})


/home/tmbish/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/tmbish/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
odds.head(5)

,market_id,selection_id,selection_name,wap_5m,wap_30s,bsp,ltp,traded_vol,inplay_min_lay
0,1.179845158,23493550,1. Larmour,6.27,5.84,6.20,6.2,8277,1.19
1,1.179845158,16374800,3. Careering Away,3.31,3.67,3.60,3.65,18592,1.08
2,1.179845158,19740699,4. Bells N Bows,6.87,6.36,6.62,6.4,7413,1.42
3,1.179845158,11014075,5. Tickinover,40.73,42.75,54.42,50.0,725,24.00
4,1.179845158,21317306,6. Opera Tickets,21.54,22.38,22.22,21.0,1972,18.50


In [5]:
race.head(5)

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,selection_id,selection_name,barrier,place,trainer,jockey,weight
0,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,19674744,Baldy,2,4.0,Peter Nolan,Karl Zechner,59.0
1,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,401615,Nostrovia,4,7.0,Dennis O'leary,Margaret Collett,59.0
2,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,26789410,Ammo Loco,5,1.0,John Hickmott,Barend Vorster,58.5
3,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,26623561,Gunfireinthecity,7,3.0,Dennis O'leary,Kayla Crowther,57.0
4,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,28699693,Whys,3,2.0,Trevor Day,Sairyn Fawke,57.0


In [6]:
# Joining two datasets

df = race.merge(odds.loc[:, odds.columns != 'selection_name'], how = "inner", on = ['market_id', 'selection_id'])

#df = df[['date', 'market_id', 'selection_id', 'country', 'track', 'race_type', 'race_number', 'start_time', 'weather', 'track_condition', 'race_distance', 'barrier', '']]
df

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,...,place,trainer,jockey,weight,wap_5m,wap_30s,bsp,ltp,traded_vol,inplay_min_lay
0,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,4.0,Peter Nolan,Karl Zechner,59.0,3.30,3.76,3.49,3.5,16543,2.70
1,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,7.0,Dennis O'leary,Margaret Collett,59.0,45.00,72.01,102.90,100.0,583,130.00
2,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,1.0,John Hickmott,Barend Vorster,58.5,2.80,2.46,2.74,3.0,36043,1.01
3,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,3.0,Dennis O'leary,Kayla Crowther,57.0,14.41,11.25,14.00,14.0,4113,7.40
4,2020-07-01,Balaklava,AUS,R,1,1.171091087,2020-07-01 02:40:00,FINE,GOOD4,2200,...,2.0,Trevor Day,Sairyn Fawke,57.0,11.37,13.58,13.62,13.5,2395,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160022,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,1.0,Sharlee D'avila,Lacey Morrison,57.5,7.22,6.48,5.97,5.9,14848,1.01
160023,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,7.0,Graham Hughes,Frank Edwards,56.0,67.74,50.79,70.42,75.0,677,75.00
160024,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,4.0,Alby Molino,M Potgieter,55.0,12.55,11.02,13.24,14.0,6700,10.00
160025,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,6.0,Chris Parry,Wanderson D'avila,55.0,7.09,7.51,8.78,9.0,8776,4.20


In [7]:
df.date.value_counts()

2021-04-03    1168
2020-10-17    1164
2021-05-29    1158
2020-11-03    1111
2021-05-22    1107
              ... 
2020-12-21     100
2021-03-22      95
2020-10-26      85
2021-03-15      62
2021-01-05      51
Name: date, Length: 351, dtype: int64

In [8]:
df.barrier.value_counts()

2     14336
1     14328
3     14310
4     14290
5     14237
6     13970
7     13484
8     12642
9     11245
10     9628
11     7820
12     5996
13     4426
14     3219
15     2265
16     1583
17     1042
18      678
19      283
20      167
21       45
22       20
23        8
24        5
Name: barrier, dtype: int64

In [10]:
df.race_distance.value_counts()

1200    15873
1400    11665
1200    10914
1000     9584
1600     8620
        ...  
1510        5
4000        5
2015        5
2206        5
1845        4
Name: race_distance, Length: 564, dtype: int64

In [11]:
# Functions
# __________________

def distance_group(distance):


    if distance is None:
        return("missing")
    elif distance < 1100:
        return("sprint")
    elif distance < 1400:
        return("mid_short")
    elif distance < 1800:
        return("mid_long")
    else:
        return("long")

def barrier_group(barrier):
    if barrier is None:
        return("missing")
    elif barrier < 4:
        return("inside")
    elif barrier < 9:
        return("mid_field")
    else:
        return("outside")


In [12]:
df['distance_group'] = pd.to_numeric(df.race_distance, errors = "coerce").apply(distance_group)
df['barrier_group'] = pd.to_numeric(df.barrier, errors = "coerce").apply(barrier_group)

In [13]:
# Apply Back Profit Less Commission

df['back_npl'] = np.where(df['place'] == 1, 0.95 * (df['bsp']-1), -1)
df['lay_npl'] = np.where(df['place'] == 1, -1 * (df['bsp']-1), 0.95)

# df.query('place == 1')[['bsp', 'place', 'back_npl']]


In [1]:
df

NameError: name 'df' is not defined

In [14]:
df.distance_group.value_counts()

mid_short    64998
mid_long     51038
sprint       23171
long         20820
Name: distance_group, dtype: int64

In [15]:
df.barrier_group.value_counts()

mid_field    68623
outside      48430
inside       42974
Name: barrier_group, dtype: int64

In [16]:
df_train = df.query('date < "2021-04-01"')
df_test = df.query('date >= "2021-04-01"')

In [17]:
'{} rows in the "training" set and {} rows in the "test" data'.format(df_train.shape[0], df_test.shape[0])

'119244 rows in the "training" set and 40783 rows in the "test" data'

# SIMPLE

Track / Barrier / Distance

In [18]:

# Simple

# Bet placement: BSP Back or Lay
# Strategy: Select a profitable track / distance / barrier group

trackDistanceBarrier = (
    df_train
    .assign(stake = 1)
    .assign(odds = lambda x: x['bsp'])
    .groupby(['track', 'race_distance', 'barrier_group'], as_index=False)
    .agg({'back_npl': 'sum', 'lay_npl': 'sum','stake': 'sum', 'odds': 'mean'})
)

trackDistanceBarrier



,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds
0,Albany,1000,inside,11.2550,-11.95,2,15.450000
1,Albany,1000,mid_field,-5.0000,4.75,5,101.136000
2,Albany,1000,outside,-5.0000,4.75,5,88.374000
3,Albany,1100,inside,-3.0525,2.70,6,29.430000
4,Albany,1100,mid_field,-6.4040,5.92,9,37.483333
...,...,...,...,...,...,...,...
6325,York,1500,inside,1.8995,-2.41,6,41.195000
6326,York,1500,mid_field,-7.0000,6.65,7,32.472857
6327,York,1920,inside,-3.0000,2.85,3,21.883333
6328,York,1920,mid_field,-0.3520,-0.04,5,20.978000


In [36]:
def pl_pValue(number_bets, npl, stake, average_odds):

    pot = npl / stake

    tStatistic = (pot * np.sqrt(number_bets)) / np.sqrt( (1 + pot) * (average_odds - 1 - pot) )

    pValue = 2 * t.cdf(-abs(tStatistic), number_bets-1)

    return(np.where(np.logical_or(np.isnan(pValue), pValue == 0), 1, pValue))


trackDistanceBarrier = (
    trackDistanceBarrier
    .assign(backPL_pValue = lambda x: pl_pValue(number_bets = x['stake'], npl = x['back_npl'], stake = x['stake'], average_odds = x['odds']))
    .assign(layPL_pValue = lambda x: pl_pValue(number_bets = x['stake'], npl = x['lay_npl'], stake = x['stake'], average_odds = x['odds']))
)

trackDistanceBarrier.sort_values("backPL_pValue")

/home/tmbish/.local/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,backPL_pValue,layPL_pValue
2487,Goulburn,1300,inside,-36.4965,34.62,38,28.381842,0.000002,0.434784
4835,Randwick,1000,outside,-45.1355,42.79,47,48.943617,0.000022,0.513528
3619,Moonee Valley,1200,inside,-52.2195,48.81,64,17.725313,0.000565,0.254399
1908,Flemington,1600,mid_field,405.4345,-432.11,60,37.159333,0.000992,1.000000
534,Belmont,1650,inside,-21.3445,20.21,23,19.891739,0.001173,0.476452
...,...,...,...,...,...,...,...,...,...
3237,Kyneton,1218,inside,-3.0000,2.85,3,25.293333,1.000000,0.830056
3242,Kyneton,1450,outside,-0.7885,-1.12,21,NaN,1.000000,1.000000
3243,Kyneton,1469,inside,3.6005,-3.79,1,4.790000,1.000000,1.000000
3210,Kilmore,1462,inside,-5.0000,4.75,5,25.054000,1.000000,0.767465


In [39]:
trackDistanceBarrier.sort_values("backPL_pValue").query('back_npl > 0')

,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,backPL_pValue,layPL_pValue
1908,Flemington,1600,mid_field,405.4345,-432.11,60,37.159333,0.000992,1.0
5108,Sale,1100,outside,630.4670,-664.06,7,124.254286,0.004904,1.0
4528,Pakenham,1200,mid_field,572.7400,-613.25,114,69.126579,0.006916,1.0
108,Alice Springs,1600,mid_field,118.1240,-125.47,14,21.596429,0.011334,1.0
2836,Hobart,1100,outside,361.1225,-382.90,30,58.850000,0.011511,1.0
...,...,...,...,...,...,...,...,...,...
324,Ballarat,2600,inside,8.8730,-9.34,1,10.340000,1.000000,1.0
4029,Mudgee,1600,outside,10.2315,-10.77,1,11.770000,1.000000,1.0
4312,Newcastle,2900,outside,1.1685,-1.23,1,2.230000,1.000000,1.0
2477,Gosford,2600,outside,10.4500,-11.00,1,12.000000,1.000000,1.0


In [42]:
# Test Top 5 Lay Strategies
bestLay = trackDistanceBarrier.query('lay_npl>0').sort_values('layPL_pValue').head(5)
bestLay


,track,race_distance,barrier_group,back_npl,lay_npl,stake,odds,backPL_pValue,layPL_pValue
188,Ascot,1000,inside,-83.6395,77.06,115,24.616870,0.003054,0.248157
3619,Moonee Valley,1200,inside,-52.2195,48.81,64,17.725313,0.000565,0.254399
6299,Yeppoon,1400,inside,-11.0000,10.45,11,6.022727,1.000000,0.289686
959,Caulfield,1400,mid_field,-74.3150,67.45,114,24.828772,0.018780,0.301137
1366,Darwin,1200,mid_field,-47.3980,43.94,64,19.354531,0.009844,0.318178


In [44]:
trainSetTopLay = (
    df_train
    .merge(layBest[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
)

testSetTopLay = (
    df_test
    .merge(layBest[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
)

testSetTopLay

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,...,bsp,ltp,traded_vol,inplay_min_lay,distance_group,barrier_group,back_npl,lay_npl,npl,stake
0,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,11.08,11.5,4302,8.80,sprint,mid_field,-1.0000,0.95,0.95,1
1,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,5.41,5.4,22583,5.00,sprint,outside,-1.0000,0.95,0.95,1
2,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,4.73,4.6,33321,1.01,sprint,inside,3.5435,-3.73,-3.73,1
3,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,7.35,7.2,9892,7.00,sprint,mid_field,-1.0000,0.95,0.95,1
4,2021-04-02,Ascot,AUS,R,6,1.181330624,2021-04-02 07:15:00,None,None,1000,...,4.97,4.9,15590,4.80,sprint,outside,-1.0000,0.95,0.95,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,18.31,19.5,4171,19.50,mid_short,outside,-1.0000,0.95,0.95,1
220,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,42.00,40.0,1118,44.00,mid_short,mid_field,-1.0000,0.95,0.95,1
221,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,29.91,30.0,1090,9.60,mid_short,mid_field,-1.0000,0.95,0.95,1
222,2021-06-26,Darwin,AUS,R,3,1.184733965,2021-06-26 05:37:00,None,None,1200,...,6.60,6.4,14863,1.01,mid_short,outside,5.3200,-5.60,-5.60,1


In [45]:
def bet_eval_metrics(d, side = False):

    if side:
        metrics = (d
         .groupby('bet_side', as_index=False)
         .agg({"npl": "sum", "stake": "sum"})
         .assign(pot=lambda x: x['npl'] / x['stake'])
        )
    else:
        metrics = pd.DataFrame(d
         .agg({"npl": "sum", "stake": "sum"})
        ).transpose().assign(pot=lambda x: x['npl'] / x['stake'])

    return(metrics[metrics['stake'] != 0])

def bet_eval_chart_cPl(d):

    d = (
        d
        .groupby('market_id')
        .agg({'npl': 'sum'})
    )

    d['market_number'] = np.arange(len(d))
    d['cNpl'] = d.npl.cumsum()

    chart = px.line(d, x="market_number", y="cNpl", title='Cumulative Net Profit', template='simple_white')

    return(chart)


In [46]:
bet_eval_metrics(testSetTopLay)


,npl,stake,pot
0,18.44,224.0,0.082321


In [49]:
# Entire Period Cumulative PL

d = (
    df_test
    .merge(layBest[['track', 'race_distance']])
    .assign(npl=lambda x: x['lay_npl'])
    .assign(stake=1)
    .groupby(['date'], as_index=False)
    .agg({'npl': 'sum'})
)    


d['cNpl'] = d.npl.cumsum()

chart = px.line(d, x="date", y="cNpl", title='Cumulative Net Profit', template='simple_white')

chart


# Medium

Are Certain Jockeys or Trainers predictable shorteners or overbet?

- Predictable shorteners (hypothesise based on 5min WAP vs 30sec WAP)
- Overbet (look at PL stat sig of lay price 30s out)

In [50]:
# Jockey Shortening Rates

(
    df_train
    .assign(market_support=lambda x: x['wap_5m'] / x['wap_30s'])
    .assign(races=1)
    .groupby('jockey')
    .agg({'market_support': 'mean', 'races': 'count'})
    .query('races > 10')
    .sort_values('market_support', ascending = False)
)


,market_support,races
jockey,,
Scott Sheargold,1.133095,192
Lorelle Crow,1.056582,106
Chris Mc Carthy,1.051022,26
Anthony Darmanin,1.048931,142
James Winks,1.048893,12
...,...,...
Cyril Small,0.731661,16
A Kajiya,0.720798,21
J Luxe,0.710535,11


In [66]:
# Group By Jockey and Market Support
jockeys = (
    df_train
    .assign(stake = 1)
    .assign(odds = lambda x: x['bsp'])
    .assign(npl=lambda x: np.where(x['place'] == 1, 0.95 * (x['odds']-1), -1))
    .assign(market_support=lambda x: np.where(x['wap_5m'] > x['wap_30s'], "Y", "N"))
    .groupby(['jockey', 'market_support'], as_index=False)
    .agg({'odds': 'mean', 'stake': 'sum', 'npl': 'sum'})
    .assign(pValue = lambda x: pl_pValue(number_bets = x['stake'], npl = x['npl'], stake = x['stake'], average_odds = x['odds']))
)

jockeys.sort_values('pValue').query('npl > 0').head(10)


,jockey,market_support,odds,stake,npl,pValue
624,K Jennings,Y,18.106118,85,178.6955,0.005643
496,Jade Darose,Y,87.343333,39,579.4265,0.008942
263,Clayton Gallagher,Y,24.225338,148,226.7145,0.012994
906,Ms T Harrison,Y,27.944125,160,241.3065,0.018095
615,Justin P Stanley,N,13.084502,231,155.7305,0.019913
802,Michael Dee,N,36.338213,263,299.7255,0.031634
753,Madeleine Wishart,Y,25.249872,78,156.5065,0.033329
433,Hannah Fitzgerald,Y,32.171944,72,170.1830,0.045334
937,Nick Heywood,N,17.172857,98,111.8905,0.049176
745,M Pateman,N,22.690808,260,189.2050,0.052283


In [67]:
bestJockey = (
    df_test
    .assign(market_support=lambda x: np.where(x['wap_5m'] > x['wap_30s'], "Y", "N"))
    .merge(jockeys.sort_values('pValue').query('npl > 0').head(10)[['jockey', 'market_support']])
    .assign(stake = 1)
    .assign(odds = lambda x: x['bsp'])
    .assign(npl=lambda x: np.where(x['place'] == 1, 0.95 * (x['odds']-1), -1))
)

bet_eval_metrics(bestJockey)


,npl,stake,pot
0,36.329,375.0,0.096877


In [68]:
bet_eval_chart_cPl(bestJockey)

# Hard

Do certain jockeys or certain jockeys under certain track conditions present back to lay opportunities?

The way this would work is you'd have some target

In [162]:
# First Investigate The Average Inplay Minimums And Loss Rates of Certain Jockeys
tradeOutIndex = (
    df_train
    .query('bsp < 20')
    .assign(inplay_odds_ratio=lambda x: x['inplay_min_lay'] / x['bsp'])
    .assign(win=lambda x: np.where(x['place']==1,1,0))
    .assign(races=lambda x: 1)
    # .groupby('trainer', as_index = False)
    .groupby(['trainer', 'jockey'], as_index=False)
    .agg({'inplay_odds_ratio': 'mean', 'win': 'mean', 'races': 'sum'})
    .assign(tradeOutIndex = lambda x: np.sqrt(x['inplay_odds_ratio'] + x['win']))
    .sort_values('tradeOutIndex')
    .query('races >= 10')
)

tradeOutIndex.head(5)


,trainer,jockey,inplay_odds_ratio,win,races,tradeOutIndex
18171,Wayne Walters,Kate Walters,0.466679,0.00,17,0.683139
12071,Mitchell Freedman,Billy Egan,0.512664,0.00,11,0.716006
10336,M & L Cerchi,Jordan Childs,0.553930,0.00,11,0.744265
1112,B & P & M Joseph & Jones,Patrick Scorse,0.526690,0.04,25,0.752788
5385,Gary Duncan,Taylor Marshall,0.570788,0.00,12,0.755505


In [199]:
targetTradeoutFraction = 0.7

betSet = (
    df_test
    .query('bsp < 10')
    .merge(tradeOutIndex.head(50)[['trainer', 'jockey']])
    .assign(stake=lambda x: 1)
    .assign(npl=lambda x: np.where(x['inplay_min_lay'] <= targetTradeoutFraction * x['bsp'], np.where(x['place'] == 1, 0.95 * (x['bsp']-1-1), 0), -1))
)

betSet[['place', 'bsp', 'inplay_min_lay', 'npl']]

#bet_eval_metrics(betSet)

,place,bsp,inplay_min_lay,npl
0,4.0,7.18,5.20,-1.000
1,1.0,4.40,1.65,2.280
2,2.0,5.42,3.55,0.000
3,3.0,6.83,7.00,-1.000
4,6.0,9.87,8.60,-1.000
...,...,...,...,...
83,5.0,3.90,3.55,-1.000
84,4.0,8.92,8.00,-1.000
85,1.0,2.06,1.01,0.057
86,NaN,7.20,5.90,-1.000


In [200]:

betSet.npl.value_counts()



 0.0000    37
-1.0000    36
 1.1875     2
 2.2800     1
 1.5200     1
 0.1330     1
 2.0140     1
 2.6410     1
 2.4700     1
 1.3965     1
 1.8240     1
-0.1805     1
 2.5650     1
 0.6745     1
 1.4915     1
 0.0570     1
Name: npl, dtype: int64

In [201]:
bet_eval_metrics(betSet)

,npl,stake,pot
0,-14.739,88.0,-0.167489


In [202]:
df_test[['bsp', 'inplay_min_lay']]

,bsp,inplay_min_lay
119244,14.01,7.80
119245,6.00,5.80
119246,30.58,29.00
119247,2.08,1.50
119248,7.80,7.40
...,...,...
160022,5.97,1.01
160023,70.42,75.00
160024,13.24,10.00
160025,8.78,4.20


In [109]:
df_test.query('inplay_min_lay >= bsp')

,date,track,country,race_type,race_number,market_id,start_time,weather,track_condition,race_distance,...,wap_5m,wap_30s,bsp,ltp,traded_vol,inplay_min_lay,distance_group,barrier_group,back_npl,lay_npl
119255,2021-04-01,Beaudesert,AUS,R,3,1.181290317,2021-04-01 04:12:00,None,None,1200,...,73.96,98.21,95.00,90.0,286,100.00,mid_short,mid_field,-1.0,0.95
119257,2021-04-01,Beaudesert,AUS,R,3,1.181290317,2021-04-01 04:12:00,None,None,1200,...,3.98,4.14,4.30,4.4,11442,4.40,mid_short,mid_field,-1.0,0.95
119274,2021-04-01,Beaudesert,AUS,R,5,1.181290321,2021-04-01 05:27:00,None,None,1650,...,8.46,6.49,6.40,6.6,6609,6.60,mid_long,inside,-1.0,0.95
119282,2021-04-01,Beaudesert,AUS,R,6,1.181290323,2021-04-01 06:04:00,None,None,1200,...,32.18,49.46,65.00,65.0,554,65.00,mid_short,mid_field,-1.0,0.95
119284,2021-04-01,Beaudesert,AUS,R,6,1.181290323,2021-04-01 06:04:00,None,None,1200,...,42.50,27.36,38.00,36.0,664,38.00,mid_short,inside,-1.0,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159986,2021-06-29,Townsville,AUS,R,1,1.184845386,2021-06-29 03:57:00,None,None,1000,...,44.27,50.89,55.00,55.0,790,55.00,sprint,inside,-1.0,0.95
159998,2021-06-29,Townsville,AUS,R,3,1.184845390,2021-06-29 05:12:00,None,None,1200,...,37.37,30.51,32.00,30.0,1020,36.00,mid_short,inside,-1.0,0.95
160004,2021-06-29,Townsville,AUS,R,4,1.184845392,2021-06-29 05:47:00,None,None,1200,...,3.65,3.23,3.20,3.2,27917,3.25,mid_short,inside,-1.0,0.95
160023,2021-06-29,Townsville,AUS,R,6,1.184845396,2021-06-29 07:00:00,None,None,1000,...,67.74,50.79,70.42,75.0,677,75.00,sprint,inside,-1.0,0.95
